In [1]:
import numpy as np
import pandas as pd
import pulp

# The Maximal Flow Problem

<img src="04 - The Maximal Flow Problem image 01.png">

Due to climate change, it is a particularly hot summer in Kristiansand this year. The citizens, who have all gathered for the Quart 3.0 festival outside the Christiansholm Fortress, are all thirste for some bear, but the event has not been able to get hold of more from the local brewery Christiansand Bryggeri (CB) because of a blockade made by the religious, self-appointed police force called The Morality Police. Luckily, the city has already anticipted this when they long ago decided to secretly build an undergorund network of pipes that can transport CB beer from the brfewery to any tavern in the city. The Quart 3.0 commitee therefor calls CB and pleads that they send over as much beer as they can. However, how do we maximize get the maximum amount of beer to from the brewery to the festival through this complicated web of pipes? This is a perfect area of use for The Maximal Flow Problem.

The image above shows an illustration of this problem as a graph consisting of nodes and edges. In the image, the maximum flow of beer _out_ from a node is written along its edge.

Finally, notice that due to different quality of the valves and motor pumps used for transporting the beer, some nodes are not able to deliver out as much as they are able to receive. Some nodes are not even able to send out beer over an edge even though is is able to receive, as in `(D,E)` vs. `(E,D)`. The model must account for this as well.

## Prepare the data

In [2]:
# Constants
Nodes = ['A','B','C','D','E','F','G','H','I','J']
Transshipment_nodes = ['B','C','D','F','G','H','I','J']
Sink = 'A'
Source = 'E'

In [3]:
# Maximum flow
data = [
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [   230, np.nan,    200,    160, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [   240,    180, np.nan, np.nan,    140, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan,    250, np.nan, np.nan, np.nan,    110, np.nan, np.nan, np.nan, np.nan],
    [np.nan, np.nan, np.nan,    300, np.nan, np.nan,    400, np.nan, np.nan, np.nan],
    [np.nan, np.nan,    100,     80, np.nan, np.nan,    130, np.nan,    110, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan,    180, np.nan,    200, np.nan, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,     90, np.nan,    100, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan,     60, np.nan,     90, np.nan,     90],
    [   120, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,    110, np.nan]
]

max_flow_df = pd.DataFrame(data=data, index=Nodes, columns=Nodes)
max_flow_df.fillna(' ')

,A,B,C,D,E,F,G,H,I,J
A,,,,,,,,,,
B,230,,200,160,,,,,,
C,240,180,,,140,,,,,
D,,250,,,,110,,,,
E,,,,300,,,400,,,
F,,,100,80,,,130,,110,
G,,,,,,180,,200,,
H,,,,,,,90,,100,
I,,,,,,60,,90,,90
J,120,,,,,,,,110,


## Create the variables

We define the variable $x_{i,j}$ to be a continous variabel that describe the flow from node $i$ to node $j$. The value of each variable can vary between no flow (zero) and the maximum flow that the edge can provide. Since any node is not connected directly to all the other nodes, we only need to create variables for the connections that actually exsist. If you count, you will see that there are 14 different routes on the map. Further, since each route between any two nodes should support sending a flow in both directions, we need two variables per edge (one for each direction). This mean that we end up with a total of 28 variables to describe this problem.

In [4]:
# Let us first extract only the values the correspond to a route between to nodes and save them as a dictionary
max_flow = {i:{j:flow for j, flow in row.items() if pd.notnull(flow)} for i, row in max_flow_df.T.to_dict().items()}
max_flow

{'A': {},
 'B': {'A': 230.0, 'C': 200.0, 'D': 160.0},
 'C': {'A': 240.0, 'B': 180.0, 'E': 140.0},
 'D': {'B': 250.0, 'F': 110.0},
 'E': {'D': 300.0, 'G': 400.0},
 'F': {'C': 100.0, 'D': 80.0, 'G': 130.0, 'I': 110.0},
 'G': {'F': 180.0, 'H': 200.0},
 'H': {'G': 90.0, 'I': 100.0},
 'I': {'F': 60.0, 'H': 90.0, 'J': 90.0},
 'J': {'A': 120.0, 'I': 110.0}}

A formal LP formulation would normally include a constraint for the maximum flow over each edge, like this:
$$x_{i,j} \leq K_{i,j}$$
where $K_{i,j}$ is a constant that defines the maximum allowed flow over that edge. In PuLP, however, this can be included in the creation of the variable, where you can specify a lower and upper bound.

In [5]:
# Create variables
x = pulp.LpVariable.dicts('route',
                          ((i, j) for i in max_flow for j in max_flow[i]),
                          lowBound = 0,
                          cat='Continuous')

# Limit maximum flow
for i in max_flow:
    for j in max_flow[i]:
        x[i,j].upBound = max_flow[i][j]

## Initiate an empty LP Problem

In [6]:
prob = pulp.LpProblem("MaximalFlowProblem", pulp.LpMaximize)

## Create the constraints

### Flow to sink (Quart 3.0) must equal flow from source (CB)

Since we ca trust the taverns (nodes) in the networks to not store any of the beer for themselves, we can assume that the amount of beer sent out of the bewery source node `E` equals the amount of beer received at the festival sink node `A`.
$$x_{E,D} + x_{E,G} = x_{B,A} + x_{C,A} + x_{J,A}$$

or simply

$$\textrm{flow out from source} = \textrm{flow in to sink}$$

In [7]:
from_source = pulp.lpSum([x[Source, j] for j in max_flow_df.T[Source].dropna().keys()])
to_sink = pulp.lpSum([x[i, Sink] for i in max_flow_df[Sink].dropna().keys()])

prob += from_source == to_sink, f"From source {Source} to sink {Sink}"

### What goes in must come out

Similarly to the transshipment problem, we can set up a constraint for all tavern transshipment nodes (`B`,`C`,`D`,`F`,`G`,`H`,`I`,`J`) that all the beer flow enetering this node should also leave it. As an example, the rule for the transshipment node `H` would be:
$$x_{G,H} + x_{I,H} = x_{H,G} + x_{H,I}$$

or simply

$$\textrm{flow in} = \textrm{flow out}$$

In [8]:
for i in Transshipment_nodes:
    node_out = pulp.lpSum([x[i,j] for j in max_flow_df.loc[i,:].dropna().index])
    node_in = pulp.lpSum([x[j,i] for j in max_flow_df.loc[:,i].dropna().index])
    
    prob += node_in == node_out, f"Flow in and out from node {i}"

## Create the objective function
Since we want to maximize the amount of beer flowing to node `A`, this is the same as maximizing the flow over every edge going into node `A`.

In [9]:
max_flow_df[Source].dropna()

C    140.0
Name: E, dtype: float64

In [10]:
max_flow_df.fillna(' ')

,A,B,C,D,E,F,G,H,I,J
A,,,,,,,,,,
B,230,,200,160,,,,,,
C,240,180,,,140,,,,,
D,,250,,,,110,,,,
E,,,,300,,,400,,,
F,,,100,80,,,130,,110,
G,,,,,,180,,200,,
H,,,,,,,90,,100,
I,,,,,,60,,90,,90
J,120,,,,,,,,110,


In [11]:
prob += pulp.lpSum([x[i,Sink] for i in max_flow_df[Sink].dropna().keys()])

## Find the optimal solution

In [12]:
prob.solve()
status = pulp.LpStatus[prob.status]
obj_value = prob.objective.value()

print(f"The solver found a solution that is *{status}*, where the total amount of beer reveiced was {obj_value:,.1f} liters")

The solver found a solution that is *Optimal*, where the total amount of beer reveiced was 440.0 liters


In [13]:
results = pd.DataFrame.from_dict(data={i: {j: x[i,j].value() for j in max_flow[i]} for i in max_flow}, orient='index')
results = pd.concat([pd.DataFrame([], index=['A']), results], sort=True)

results.fillna('')

,A,B,C,D,E,F,G,H,I,J
A,,,,,,,,,,
B,230,,20,0,,,,,,
C,120,0,,,0,,,,,
D,,250,,,,50,,,,
E,,,,300,,,140,,,
F,,,100,0,,,0,,0,
G,,,,,,50,,90,,
H,,,,,,,0,,90,
I,,,,,,0,,0,,90
J,90,,,,,,,,0,


The final solutions is represented in the table above, where each number represents how much beer will be transported from node $i$ to node $j$. The same result is summarized in the image below. Notice that for this example there are multiple global optimums, and your results when running this notebook might be slightly different than this solution (though the objective value should stay the same). The reason for this is because there exists other, equally good solutions for this problem, and it can be kind of random what solution your solver finds first. An equally good solution can be observed at node `C`, where our solution chooses to send 20 liters of beer from `C` to `B` before sending it to node `A`. You would have an equally good solution if node `C` just sent those 20 liters directly to `A`.

When stating that the alternative solution above is "equally good", it means that it generates an equally good objective value. However, in the case of transpoting units through a physical network, it is often desirable to also minimize either the number of nodes the units pass through or the totalt work that has to be done to process the total network flow. So there might axists solutions that are better or worse for the actual problem you want to solve, but give a similar, or even worse, objective value as a result of how you have formulated your problem. If this is the case, you would normally have to add more constraints or tweak your objective function to make the formulation better support your desired behaviour of the system.

<img src="04 - The Maximal Flow Problem image 02.png">